# Neural Networks for Binary Classification\n\n**Multi-Layer Perceptron Implementation from Scratch**\n\nThis notebook demonstrates the implementation and comparison of MLP classifiers:\n- Custom NumPy-based MLP with SGD optimization\n- Adam optimizer implementation with early stopping\n- Comparison with scikit-learn and PyTorch implementations\n\nDataset: Vehicle Purchase Quality Prediction (\"Don't Get Kicked!\" from Kaggle)

In [1]:
!pip install -q pandas numpy matplotlib scikit-learn torch

---
## 1. Data Preparation and Temporal Split

In [2]:
# Импорт всех необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Фиксация random seed для воспроизводимости
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f'PyTorch version: {torch.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

PyTorch version: 2.9.0+cu126
NumPy version: 2.0.2
CUDA available: True
GPU: Tesla T4


In [3]:
# Загрузка данных
data = pd.read_csv('data/training.csv', index_col=0, parse_dates=['PurchDate'])
print(f'Размер датасета: {data.shape}')
data.info()

Размер датасета: (72983, 33)
<class 'pandas.core.frame.DataFrame'>
Index: 72983 entries, 1 to 73014
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   IsBadBuy                           72983 non-null  int64         
 1   PurchDate                          72983 non-null  datetime64[ns]
 2   Auction                            72983 non-null  object        
 3   VehYear                            72983 non-null  int64         
 4   VehicleAge                         72983 non-null  int64         
 5   Make                               72983 non-null  object        
 6   Model                              72983 non-null  object        
 7   Trim                               70623 non-null  object        
 8   SubModel                           72975 non-null  object        
 9   Color                              72975 non-null  object        
 10  Transmissi

In [4]:
# Разделение признаков на категориальные и числовые
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
numerical_cols = data.select_dtypes(include=['number']).columns.tolist()
numerical_cols.remove('IsBadBuy')  # Убираем целевую переменную

print(f'Числовые признаки ({len(numerical_cols)}): {numerical_cols}')
print(f'\nКатегориальные признаки ({len(categorical_cols)}): {categorical_cols}')

Числовые признаки (17): ['VehYear', 'VehicleAge', 'WheelTypeID', 'VehOdo', 'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice', 'BYRNO', 'VNZIP1', 'VehBCost', 'IsOnlineSale', 'WarrantyCost']

Категориальные признаки (14): ['Auction', 'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission', 'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName', 'PRIMEUNIT', 'AUCGUART', 'VNST']


In [5]:
def temporal_split(df, train_ratio=1/3, valid_ratio=1/3):
    """
    Разбиение датасета по времени:
    train.PurchDate < valid.PurchDate < test.PurchDate

    Первые 33% записей (по дате) — train, средние 33% — valid, последние 33% — test.
    """
    df = df.copy()
    df['PurchDate'] = pd.to_datetime(df['PurchDate'])
    df = df.sort_values(by='PurchDate').reset_index(drop=True)

    n = len(df)
    split1_idx = int(n * train_ratio)
    split2_idx = int(n * (train_ratio + valid_ratio))

    df_train = df.iloc[:split1_idx].reset_index(drop=True)
    df_valid = df.iloc[split1_idx:split2_idx].reset_index(drop=True)
    df_test = df.iloc[split2_idx:].reset_index(drop=True)

    print('Разбиение датасета (train/valid/test):')
    print(f'  Train: {len(df_train)/len(df)*100:.1f}% ({len(df_train)} samples)')
    print(f'  Valid: {len(df_valid)/len(df)*100:.1f}% ({len(df_valid)} samples)')
    print(f'  Test:  {len(df_test)/len(df)*100:.1f}% ({len(df_test)} samples)')
    print(f'\nДиапазоны дат:')
    print(f'  Train: {df_train["PurchDate"].min()} — {df_train["PurchDate"].max()}')
    print(f'  Valid: {df_valid["PurchDate"].min()} — {df_valid["PurchDate"].max()}')
    print(f'  Test:  {df_test["PurchDate"].min()} — {df_test["PurchDate"].max()}')

    return df_train, df_valid, df_test


# Разбиение данных по времени
df_train, df_valid, df_test = temporal_split(data)

Разбиение датасета (train/valid/test):
  Train: 33.3% (24327 samples)
  Valid: 33.3% (24328 samples)
  Test:  33.3% (24328 samples)

Диапазоны дат:
  Train: 2009-01-05 00:00:00 — 2009-09-15 00:00:00
  Valid: 2009-09-15 00:00:00 — 2010-05-14 00:00:00
  Test:  2010-05-14 00:00:00 — 2010-12-30 00:00:00


In [6]:
def prepare_features(df_train, df_valid, df_test, cat_cols, num_cols):
    """
    Подготовка признаков: заполнение пропусков, кодирование категориальных переменных,
    масштабирование числовых признаков.
    """
    # Извлекаем целевую переменную
    y_train = df_train['IsBadBuy'].values
    y_valid = df_valid['IsBadBuy'].values
    y_test = df_test['IsBadBuy'].values

    # Убираем целевую переменную и дату из признаков
    X_train = df_train.drop(columns=['IsBadBuy', 'PurchDate'])
    X_valid = df_valid.drop(columns=['IsBadBuy', 'PurchDate'])
    X_test = df_test.drop(columns=['IsBadBuy', 'PurchDate'])

    # Заполнение пропусков: категориальные — 'missing'
    for col in cat_cols:
        X_train[col] = X_train[col].fillna('missing').astype(str)
        X_valid[col] = X_valid[col].fillna('missing').astype(str)
        X_test[col] = X_test[col].fillna('missing').astype(str)

    # Числовые: заполнение медианой из train
    num_imputer = SimpleImputer(strategy='median')
    X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
    X_valid[num_cols] = num_imputer.transform(X_valid[num_cols])
    X_test[num_cols] = num_imputer.transform(X_test[num_cols])

    # One-Hot Encoding для категориальных признаков
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_train_cat = encoder.fit_transform(X_train[cat_cols])
    X_valid_cat = encoder.transform(X_valid[cat_cols])
    X_test_cat = encoder.transform(X_test[cat_cols])

    # Масштабирование числовых признаков
    scaler = StandardScaler()
    X_train_num = scaler.fit_transform(X_train[num_cols])
    X_valid_num = scaler.transform(X_valid[num_cols])
    X_test_num = scaler.transform(X_test[num_cols])

    # Объединение признаков: числовые + категориальные
    X_train_processed = np.hstack([X_train_num, X_train_cat])
    X_valid_processed = np.hstack([X_valid_num, X_valid_cat])
    X_test_processed = np.hstack([X_test_num, X_test_cat])

    print(f'Размерность после предобработки:')
    print(f'  X_train: {X_train_processed.shape}')
    print(f'  X_valid: {X_valid_processed.shape}')
    print(f'  X_test:  {X_test_processed.shape}')

    return X_train_processed, X_valid_processed, X_test_processed, y_train, y_valid, y_test


# Подготовка признаков
X_train, X_valid, X_test, y_train, y_valid, y_test = prepare_features(
    df_train, df_valid, df_test, categorical_cols, numerical_cols
)

Размерность после предобработки:
  X_train: (24327, 1716)
  X_valid: (24328, 1716)
  X_test:  (24328, 1716)


In [7]:
# Проверка распределения целевой переменной
print('Распределение IsBadBuy:')
print(f'  Train: {y_train.mean():.4f} ({y_train.sum()} / {len(y_train)})')
print(f'  Valid: {y_valid.mean():.4f} ({y_valid.sum()} / {len(y_valid)})')
print(f'  Test:  {y_test.mean():.4f} ({y_test.sum()} / {len(y_test)})')

Распределение IsBadBuy:
  Train: 0.1149 (2794 / 24327)
  Valid: 0.1301 (3164 / 24328)
  Test:  0.1241 (3018 / 24328)


In [8]:
def compute_gini(y_true, y_prob):
    """Вычисление коэффициента Gini: Gini = 2 * AUC - 1"""
    auc = roc_auc_score(y_true, y_prob)
    return 2 * auc - 1

---
## 2. MLP Implementation from Scratch

In [9]:
class ActivationFunctions:
    """
    Класс с реализациями функций активации и их производных.
    """

    @staticmethod
    def sigmoid(x):
        """Сигмоидная функция: σ(x) = 1 / (1 + exp(-x))"""
        x = np.clip(x, -500, 500)
        return 1.0 / (1.0 + np.exp(-x))

    @staticmethod
    def sigmoid_derivative(x):
        """Производная sigmoid: σ'(x) = σ(x) * (1 - σ(x))"""
        s = ActivationFunctions.sigmoid(x)
        return s * (1 - s)

    @staticmethod
    def relu(x):
        """ReLU: max(0, x)"""
        return np.maximum(0, x)

    @staticmethod
    def relu_derivative(x):
        """Производная ReLU: 1 если x > 0, иначе 0"""
        return (x > 0).astype(float)

    @staticmethod
    def cosine(x):
        """Косинусная активация"""
        return np.cos(x)

    @staticmethod
    def cosine_derivative(x):
        """Производная cosine: -sin(x)"""
        return -np.sin(x)

In [10]:
class MLP:
    """
    Многослойный перцептрон с 1 скрытым слоем.
    """

    def __init__(self, n_hidden=100, activation='sigmoid', learning_rate=0.01,
                 n_epochs=100, batch_size=32, random_state=42, verbose=True):
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.random_state = random_state
        self.verbose = verbose

        # Устанавливаем функцию активации
        self._set_activation(activation)

        # Веса инициализируются при вызове fit()
        self.W1 = None
        self.b1 = None
        self.W2 = None
        self.b2 = None

        # История потерь для визуализации
        self.train_losses = []
        self.valid_losses = []

    def _set_activation(self, activation):
        """
        Установка функции активации и её производной.
        """
        if callable(activation):
            self.activation = activation
            self.activation_derivative = None
            self.activation_name = getattr(activation, '__name__', 'custom')
        elif activation == 'sigmoid':
            self.activation = ActivationFunctions.sigmoid
            self.activation_derivative = ActivationFunctions.sigmoid_derivative
            self.activation_name = 'sigmoid'
        elif activation == 'relu':
            self.activation = ActivationFunctions.relu
            self.activation_derivative = ActivationFunctions.relu_derivative
            self.activation_name = 'relu'
        elif activation == 'cosine':
            self.activation = ActivationFunctions.cosine
            self.activation_derivative = ActivationFunctions.cosine_derivative
            self.activation_name = 'cosine'
        else:
            raise ValueError(f"Unknown activation: {activation}")

    def _numerical_derivative(self, x, eps=1e-7):
        """Численное приближение производной (центральная разность)."""
        return (self.activation(x + eps) - self.activation(x - eps)) / (2 * eps)

    def _initialize_weights(self, n_features):
        """
        Инициализация весов методом Xavier/Glorot.
        limit = sqrt(6 / (fan_in + fan_out))
        """
        np.random.seed(self.random_state)

        # Первый слой: n_features → n_hidden
        limit1 = np.sqrt(6.0 / (n_features + self.n_hidden))
        self.W1 = np.random.uniform(-limit1, limit1, (n_features, self.n_hidden))
        self.b1 = np.zeros((1, self.n_hidden))

        # Второй слой: n_hidden → 1
        limit2 = np.sqrt(6.0 / (self.n_hidden + 1))
        self.W2 = np.random.uniform(-limit2, limit2, (self.n_hidden, 1))
        self.b2 = np.zeros((1, 1))

    def _forward(self, X):
        """Прямой проход через сеть."""
        # Скрытый слой
        z1 = X @ self.W1 + self.b1           # Линейная комбинация
        a1 = self.activation(z1)              # Активация

        # Выходной слой
        z2 = a1 @ self.W2 + self.b2          # Линейная комбинация
        a2 = ActivationFunctions.sigmoid(z2)  # Sigmoid для вероятности

        # Сохраняем промежуточные значения для backpropagation
        cache = {'X': X, 'z1': z1, 'a1': a1, 'z2': z2, 'a2': a2}
        return cache

    def _compute_loss(self, y_true, y_pred):
        """Binary Cross Entropy Loss."""
        if y_pred.ndim == 2:
            y_pred = y_pred[:, 1] if y_pred.shape[1] == 2 else y_pred.flatten()

        eps = 1e-15  # Защита от log(0)
        y_pred = np.clip(y_pred, eps, 1 - eps)
        loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return loss

    def _backward(self, y_true, cache):
        """Обратное распространение ошибки (backpropagation)."""
        m = y_true.shape[0]  # Размер батча
        y_true = y_true.reshape(-1, 1)

        # Извлекаем значения из cache
        X = cache['X']
        z1 = cache['z1']
        a1 = cache['a1']
        a2 = cache['a2']

        # Градиент для выходного слоя
        dz2 = a2 - y_true                              # Ошибка выхода
        dW2 = (1/m) * (a1.T @ dz2)                     # Градиент весов W2
        db2 = (1/m) * np.sum(dz2, axis=0, keepdims=True)  # Градиент смещения b2

        # Градиент для скрытого слоя (chain rule)
        da1 = dz2 @ self.W2.T                          # Ошибка, пришедшая в скрытый слой
        if self.activation_derivative is not None:
            dz1 = da1 * self.activation_derivative(z1)
        else:
            dz1 = da1 * self._numerical_derivative(z1)

        dW1 = (1/m) * (X.T @ dz1)                      # Градиент весов W1
        db1 = (1/m) * np.sum(dz1, axis=0, keepdims=True)  # Градиент смещения b1

        return {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2}

    def _update_weights_sgd(self, gradients):
        """Обновление весов методом SGD: w = w - lr * gradient"""
        self.W1 -= self.learning_rate * gradients['dW1']
        self.b1 -= self.learning_rate * gradients['db1']
        self.W2 -= self.learning_rate * gradients['dW2']
        self.b2 -= self.learning_rate * gradients['db2']

    def fit(self, X, y, X_valid=None, y_valid=None):
        """Обучение модели."""
        n_samples, n_features = X.shape
        self._initialize_weights(n_features)
        n_batches = int(np.ceil(n_samples / self.batch_size))

        for epoch in range(self.n_epochs):
            # Перемешиваем данные в начале каждой эпохи
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            epoch_loss = 0

            # Проходим по батчам
            for i in range(n_batches):
                start_idx = i * self.batch_size
                end_idx = min((i + 1) * self.batch_size, n_samples)

                X_batch = X_shuffled[start_idx:end_idx]
                y_batch = y_shuffled[start_idx:end_idx]

                # Forward → Loss → Backward → Update
                cache = self._forward(X_batch)
                batch_loss = self._compute_loss(y_batch, cache['a2'])
                epoch_loss += batch_loss * len(y_batch)

                gradients = self._backward(y_batch, cache)
                self._update_weights_sgd(gradients)

            epoch_loss /= n_samples
            self.train_losses.append(epoch_loss)

            # Валидация
            if X_valid is not None:
                valid_proba = self.predict_proba(X_valid)[:, 1]
                valid_loss = self._compute_loss(y_valid, valid_proba)
                self.valid_losses.append(valid_loss)

            # Вывод прогресса
            if self.verbose and (epoch + 1) % 10 == 0:
                msg = f"Epoch {epoch+1}/{self.n_epochs} - Train Loss: {epoch_loss:.4f}"
                if X_valid is not None:
                    msg += f" - Valid Loss: {valid_loss:.4f}"
                print(msg)

        return self

    def predict_proba(self, X):
        """Предсказание вероятностей."""
        cache = self._forward(X)
        prob_1 = cache['a2'].flatten()
        prob_0 = 1 - prob_1
        return np.column_stack([prob_0, prob_1])

    def predict(self, X, threshold=0.5):
        """Предсказание классов."""
        proba = self.predict_proba(X)[:, 1]
        return (proba >= threshold).astype(int)

---
## 3. Achieving Gini >= 0.15

In [11]:
# Создание и обучение модели
model = MLP(
    n_hidden=100,
    activation='sigmoid',
    learning_rate=0.01,
    n_epochs=100,
    batch_size=32,
    random_state=42,
    verbose=True
)

model.fit(X_train, y_train, X_valid, y_valid)

Epoch 10/100 - Train Loss: 0.3258 - Valid Loss: 0.3629
Epoch 20/100 - Train Loss: 0.3100 - Valid Loss: 0.3483
Epoch 30/100 - Train Loss: 0.2994 - Valid Loss: 0.3373
Epoch 40/100 - Train Loss: 0.2954 - Valid Loss: 0.3372
Epoch 50/100 - Train Loss: 0.2939 - Valid Loss: 0.3342
Epoch 60/100 - Train Loss: 0.2930 - Valid Loss: 0.3349
Epoch 70/100 - Train Loss: 0.2924 - Valid Loss: 0.3371
Epoch 80/100 - Train Loss: 0.2917 - Valid Loss: 0.3323
Epoch 90/100 - Train Loss: 0.2913 - Valid Loss: 0.3343
Epoch 100/100 - Train Loss: 0.2907 - Valid Loss: 0.3409


In [12]:
# Оценка на валидации
y_valid_proba = model.predict_proba(X_valid)[:, 1]
y_valid_pred = model.predict(X_valid)

gini_valid = compute_gini(y_valid, y_valid_proba)
print(f'Результаты на валидации:')
print(f'  Gini coefficient: {gini_valid:.4f}')
print(f'  ROC AUC: {roc_auc_score(y_valid, y_valid_proba):.4f}')
print(f'\nОтчёт классификации:')
print(classification_report(y_valid, y_valid_pred, zero_division=0))

Результаты на валидации:
  Gini coefficient: 0.4906
  ROC AUC: 0.7453

Отчёт классификации:
              precision    recall  f1-score   support

           0       0.89      0.98      0.94     21164
           1       0.64      0.21      0.31      3164

    accuracy                           0.88     24328
   macro avg       0.77      0.59      0.62     24328
weighted avg       0.86      0.88      0.85     24328



---
## 4. Comparison with sklearn MLPClassifier

In [13]:
# sklearn MLPClassifier с SGD
sklearn_mlp = MLPClassifier(
    hidden_layer_sizes=(100,),
    activation='logistic',
    solver='sgd',
    learning_rate_init=0.01,
    max_iter=100,
    batch_size=32,
    random_state=42,
    verbose=False
)

sklearn_mlp.fit(X_train, y_train)

sklearn_proba = sklearn_mlp.predict_proba(X_valid)[:, 1]
sklearn_pred = sklearn_mlp.predict(X_valid)

sklearn_gini = compute_gini(y_valid, sklearn_proba)
print('sklearn MLPClassifier (SGD):')
print(f'  Gini coefficient: {sklearn_gini:.4f}')
print(f'  ROC AUC: {roc_auc_score(y_valid, sklearn_proba):.4f}')

sklearn MLPClassifier (SGD):
  Gini coefficient: 0.4575
  ROC AUC: 0.7288


In [14]:
# Сравнительная таблица
print('Сравнение моделей:')
print(f'{"Модель":<25} {"Gini":>10}')
print('-' * 37)
print(f'{"My MLP (sigmoid)":<25} {gini_valid:>10.4f}')
print(f'{"sklearn MLP (SGD)":<25} {sklearn_gini:>10.4f}')

Сравнение моделей:
Модель                          Gini
-------------------------------------
My MLP (sigmoid)              0.4906
sklearn MLP (SGD)             0.4575


Лучше ли sklearn чем моя реализация?

Моя реализация лучше на 0.0331 Gini.

Возможные причины:

- Xavier/Glorot инициализация хорошо подходит для sigmoid
- Мой MLP — "чистый", а sklearn по умолчанию добавляет momentum=0.9 и L2-регуляризацию (alpha=0.0001), что меняет поведение алгоритма

---
## 5. Activation Function Comparison

In [15]:
fixed_params = {
    'n_hidden': 100,
    'learning_rate': 0.01,
    'n_epochs': 100,
    'batch_size': 32,
    'random_state': 42,
    'verbose': False
}

activations = ['sigmoid', 'relu', 'cosine']
activation_results = {}

print(f'Сравнение функций активации (sigmoid, ReLU, cosine)\n')
print(f'Параметры: n_hidden={fixed_params["n_hidden"]}, '
      f'lr={fixed_params["learning_rate"]}, epochs={fixed_params["n_epochs"]}\n')

for act in activations:
    mlp = MLP(activation=act, **fixed_params)
    mlp.fit(X_train, y_train, X_valid, y_valid)

    y_proba = mlp.predict_proba(X_valid)[:, 1]
    gini = compute_gini(y_valid, y_proba)

    activation_results[act] = {'gini': gini, 'model': mlp}
    print(f'{act.upper():>10}: Gini = {gini:.4f}')

best_activation = max(activation_results, key=lambda x: activation_results[x]['gini'])
print(f'\nЛучшая функция активации: {best_activation}')

Сравнение функций активации (sigmoid, ReLU, cosine)

Параметры: n_hidden=100, lr=0.01, epochs=100

   SIGMOID: Gini = 0.4906
      RELU: Gini = 0.4712
    COSINE: Gini = 0.4884

Лучшая функция активации: sigmoid


Какая функция активации лучшая?

Ответ: Sigmoid с Gini = 0.4906

Рейтинг:

- sigmoid: 0.4906
- cosine: 0.4884
- relu: 0.4712

Объяснение:

Sigmoid показывает лучший результат, вероятно потому что:

- Выход всегда от 0 до 1 — это сразу можно читать как вероятность "плохой покупки"
- Sigmoid даёт плавные (не резкие) градиенты, что помогает при большом количестве признаков (у нас 1716)
- Xavier инициализация изначально разрабатывалась именно для sigmoid, поэтому они хорошо работают вместе

---
## 6. PyTorch MLP Implementation

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используемое устройство: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Используемое устройство: cuda
GPU: Tesla T4


In [17]:
class PyTorchMLP(nn.Module):
    """MLP на PyTorch с 1 скрытым слоем."""

    def __init__(self, n_features, n_hidden=100, activation='sigmoid'):
        super(PyTorchMLP, self).__init__()

        self.fc1 = nn.Linear(n_features, n_hidden)
        self.fc2 = nn.Linear(n_hidden, 1)

        if activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'relu':
            self.activation = nn.ReLU()
        else:
            self.activation = nn.Sigmoid()

        self.output_activation = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.output_activation(x)
        return x

In [18]:
def train_pytorch_mlp(X_train, y_train, X_valid, y_valid,
                      n_hidden=100, activation='sigmoid',
                      learning_rate=0.01, n_epochs=100, batch_size=32,
                      verbose=True):
    """Функция обучения PyTorch MLP с SGD."""
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)

    # Подготовка данных для PyTorch
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Создание модели
    n_features = X_train.shape[1]
    model = PyTorchMLP(n_features, n_hidden, activation).to(device)

    # Loss и оптимизатор
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Обучение
    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0

        for X_batch, y_batch in train_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * len(y_batch)

        if verbose and (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {epoch_loss/len(X_train):.4f}')

    return model

In [19]:
print('Обучение MLP на PyTorch')

pytorch_model = train_pytorch_mlp(
    X_train, y_train, X_valid, y_valid,
    n_hidden=100,
    activation='sigmoid',
    learning_rate=0.01,
    n_epochs=100,
    batch_size=32,
    verbose=True
)

Обучение MLP на PyTorch
Epoch 20/100 - Train Loss: 0.3227
Epoch 40/100 - Train Loss: 0.2991
Epoch 60/100 - Train Loss: 0.2941
Epoch 80/100 - Train Loss: 0.2927
Epoch 100/100 - Train Loss: 0.2917


In [20]:
# Оценка PyTorch модели
pytorch_model.eval()
with torch.no_grad():
    X_valid_tensor = torch.FloatTensor(X_valid).to(device)
    valid_proba_pytorch = pytorch_model(X_valid_tensor).cpu().numpy().flatten()

pytorch_gini = compute_gini(y_valid, valid_proba_pytorch)
print(f'PyTorch MLP (SGD): Gini = {pytorch_gini:.4f}')

PyTorch MLP (SGD): Gini = 0.4893


Лучше ли PyTorch чем моя реализация?

Моя реализация лучше на 0.0013 Gini.

Возможные причины небольшого расхождения в результатах:

- Обе реализации используют одинаковый алгоритм (SGD без momentum), поэтому сравнение честное
- В нашей NumPy-реализации мы вручную настроили Xavier инициализацию под sigmoid
- NumPy считает в Float64 (высокая точность, 15 знаков после запятой), а PyTorch — в Float32 (7 знаков). За миллионы операций эта разница накапливается и даёт небольшое расхождение в результатах

Итог сравнения:

- My MLP (sigmoid): 0.4906
- PyTorch MLP: 0.4893

---
## 7. Final Evaluation on Test Data

In [21]:
print('Финальная оценка на тестовом датасете')

# Лучшая модель из сравнения активаций
best_model = activation_results[best_activation]['model']
best_model_name = f'My MLP ({best_activation})'

# Предсказания на всех датасетах
train_proba = best_model.predict_proba(X_train)[:, 1]
valid_proba = best_model.predict_proba(X_valid)[:, 1]
test_proba = best_model.predict_proba(X_test)[:, 1]

# Вычисление Gini
train_gini = compute_gini(y_train, train_proba)
valid_gini = compute_gini(y_valid, valid_proba)
test_gini = compute_gini(y_test, test_proba)

print(f'Лучшая модель: {best_model_name}')
print(f'\n{"Датасет":<12} {"Gini":>10}')
print('-' * 24)
print(f'{"Train":<12} {train_gini:>10.4f}')
print(f'{"Valid":<12} {valid_gini:>10.4f}')
print(f'{"Test":<12} {test_gini:>10.4f}')

Финальная оценка на тестовом датасете
Лучшая модель: My MLP (sigmoid)

Датасет            Gini
------------------------
Train            0.5448
Valid            0.4906
Test             0.4991


Анализ разрывов (Gap Analysis)

- Train - Valid gap: +0.0542
- Valid - Test gap: -0.0085

Наблюдается ли падение качества (Valid → Test)?

Нет, Valid и Test Gini очень близки (gap = -0.0085).
Модель хорошо обобщается на разные временные периоды.

Переобучена ли модель (overfitting)?

Ответ: Умеренное переобучение

Объяснение:

- Train Gini заметно выше Valid Gini.
- Некоторое переобучение присутствует, но в допустимых пределах.
- Модель выучивает полезные паттерны, но также немного шума.
- Может помочь лёгкая регуляризация или меньше эпох.

---
## Bonus: Adam Optimizer Implementation

In [22]:
class MLPWithAdam(MLP):
    """
    MLP с Adam оптимизатором и early stopping.
    Наследует от базового MLP класса.
    """

    def __init__(self, n_hidden=100, activation='sigmoid', learning_rate=0.001,
                 n_epochs=100, batch_size=32, random_state=42, verbose=True,
                 beta1=0.9, beta2=0.999, epsilon=1e-8,
                 patience=15, min_delta=1e-4):
        super().__init__(n_hidden, activation, learning_rate, n_epochs,
                        batch_size, random_state, verbose)

        # Adam гиперпараметры
        self.beta1 = beta1    # Коэффициент для первого момента
        self.beta2 = beta2    # Коэффициент для второго момента
        self.epsilon = epsilon  # Защита от деления на ноль

        # Early stopping параметры
        self.patience = patience
        self.min_delta = min_delta

        # Adam состояние (инициализируется при обучении)
        self.m = {}  # Первый момент (скользящее среднее градиентов)
        self.v = {}  # Второй момент (скользящее среднее квадратов градиентов)
        self.t = 0   # Счётчик шагов

        self.best_weights = None
        self.best_valid_loss = float('inf')

    def _initialize_adam_state(self):
        """Инициализация состояния Adam (нули для m и v)."""
        self.m = {
            'W1': np.zeros_like(self.W1), 'b1': np.zeros_like(self.b1),
            'W2': np.zeros_like(self.W2), 'b2': np.zeros_like(self.b2)
        }
        self.v = {
            'W1': np.zeros_like(self.W1), 'b1': np.zeros_like(self.b1),
            'W2': np.zeros_like(self.W2), 'b2': np.zeros_like(self.b2)
        }
        self.t = 0

    def _save_weights(self):
        """Сохранение текущих весов."""
        return {'W1': self.W1.copy(), 'b1': self.b1.copy(),
                'W2': self.W2.copy(), 'b2': self.b2.copy()}

    def _restore_weights(self, weights):
        """Восстановление весов."""
        self.W1 = weights['W1'].copy()
        self.b1 = weights['b1'].copy()
        self.W2 = weights['W2'].copy()
        self.b2 = weights['b2'].copy()

    def _update_weights_adam(self, gradients):
        """Обновление весов методом Adam."""
        self.t += 1

        for param_name, grad_name in [('W1', 'dW1'), ('b1', 'db1'),
                                       ('W2', 'dW2'), ('b2', 'db2')]:
            g = gradients[grad_name]

            # Обновление смещённой оценки первого момента
            self.m[param_name] = self.beta1 * self.m[param_name] + (1 - self.beta1) * g

            # Обновление смещённой оценки второго момента
            self.v[param_name] = self.beta2 * self.v[param_name] + (1 - self.beta2) * (g ** 2)

            # Коррекция смещения
            m_corrected = self.m[param_name] / (1 - self.beta1 ** self.t)
            v_corrected = self.v[param_name] / (1 - self.beta2 ** self.t)

            # Обновление параметров
            param = getattr(self, param_name)
            param -= self.learning_rate * m_corrected / (np.sqrt(v_corrected) + self.epsilon)
            setattr(self, param_name, param)

    def fit(self, X, y, X_valid=None, y_valid=None):
        """Обучение модели с Adam оптимизатором и early stopping."""
        n_samples, n_features = X.shape

        self._initialize_weights(n_features)
        self._initialize_adam_state()

        n_batches = int(np.ceil(n_samples / self.batch_size))
        epochs_without_improvement = 0

        for epoch in range(self.n_epochs):
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            epoch_loss = 0

            for i in range(n_batches):
                start_idx = i * self.batch_size
                end_idx = min((i + 1) * self.batch_size, n_samples)

                X_batch = X_shuffled[start_idx:end_idx]
                y_batch = y_shuffled[start_idx:end_idx]

                cache = self._forward(X_batch)
                batch_loss = self._compute_loss(y_batch, cache['a2'])
                epoch_loss += batch_loss * len(y_batch)

                gradients = self._backward(y_batch, cache)
                self._update_weights_adam(gradients)  # Adam вместо SGD

            epoch_loss /= n_samples
            self.train_losses.append(epoch_loss)

            # Валидация и Early Stopping
            if X_valid is not None:
                valid_pred = self.predict_proba(X_valid)[:, 1]
                valid_loss = self._compute_loss(y_valid, valid_pred)
                self.valid_losses.append(valid_loss)

                # Проверка улучшения
                if valid_loss < self.best_valid_loss - self.min_delta:
                    self.best_valid_loss = valid_loss
                    self.best_weights = self._save_weights()
                    epochs_without_improvement = 0
                else:
                    epochs_without_improvement += 1

                # Early stopping
                if epochs_without_improvement >= self.patience:
                    if self.verbose:
                        print(f"Early stopping на эпохе {epoch+1}")
                    break

            if self.verbose and (epoch + 1) % 10 == 0:
                msg = f"Epoch {epoch+1}/{self.n_epochs} - Train Loss: {epoch_loss:.4f}"
                if X_valid is not None:
                    msg += f" - Valid Loss: {valid_loss:.4f}"
                print(msg)

        # Восстановление лучших весов
        if self.best_weights is not None:
            self._restore_weights(self.best_weights)
            if self.verbose:
                print(f"\nВосстановлены лучшие веса (Valid Loss = {self.best_valid_loss:.4f})")

        return self

In [23]:
# Обучение MLP с Adam оптимизатором
print('BONUS: Реализация Adam оптимизатора')

mlp_adam = MLPWithAdam(
    n_hidden=100,
    activation='sigmoid',
    learning_rate=0.001,
    n_epochs=200,
    batch_size=32,
    random_state=42,
    verbose=True,
    patience=20
)

mlp_adam.fit(X_train, y_train, X_valid, y_valid)

BONUS: Реализация Adam оптимизатора
Epoch 10/200 - Train Loss: 0.2760 - Valid Loss: 0.3618
Epoch 20/200 - Train Loss: 0.2676 - Valid Loss: 0.4065
Early stopping на эпохе 23

Восстановлены лучшие веса (Valid Loss = 0.3380)


In [24]:
# Оценка модели с Adam
adam_proba = mlp_adam.predict_proba(X_valid)[:, 1]
adam_gini = compute_gini(y_valid, adam_proba)

print(f'My MLP (Adam): Gini = {adam_gini:.4f}')

My MLP (Adam): Gini = 0.4796
